In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%config Completer.use_jedi = False

In [101]:
from modules.evaluation import EvaluationFramework
# Ensemble Methods
from modules.ensemble import EnsembleModelParameters
from pyod.models.lscp import LSCP 
from pyod.models.feature_bagging import FeatureBagging
# Detectors
from pyod.models.iforest import IForest
from pyod.models.pca import PCA
from pyod.models.knn import KNN
from pyod.models.ocsvm import OCSVM

In [3]:
import numpy as np

def generate_data():
    x1 = np.random.multivariate_normal([10,24,1], [[5,0,0],[0,15,0],[0,0,1]], 100)
    x2 = np.random.multivariate_normal([10,24,1], [[15,0,0],[0,5,0],[0,0,2]], 100)
    x3 = np.random.multivariate_normal([10,24,1], [[5,0,0],[0,4,0],[0,0,13]], 100)
    x4 = np.array(list(zip(np.random.uniform(-50,50,10), np.random.uniform(-50,50,10), np.random.uniform(-50,50,10))))

    X = np.concatenate([x1, x2, x3, x4])
    Y = np.concatenate([np.zeros(len(x1)), np.zeros(len(x2)), np.zeros(len(x3)), np.ones(len(x4))])
    return (X,Y)

x,y = generate_data()

## EnsembleModelParameters

### Average of Estimators

In [70]:
# 5 iForest models with the same parameters
params_iforest = {
    'n_estimators':[100,100,100,100,100],
    'max_features':[1,1,1,1,1]
}

In [71]:
ens_iforest = EnsembleModelParameters(IForest, params_iforest)

In [72]:
eva_iforest = EvaluationFramework(ens_iforest)
eva_iforest.fit(x)

In [73]:
predictions = eva_iforest.predict(x)
scores = eva_iforest.score(y,predictions)

In [74]:
scores

{'ROC-AUC': 0.9683333333333333,
 'Detection-Rate': 1.0,
 'False-Alarm': 0.06333333333333334,
 'F1-Score': 0.5128205128205129,
 'Log-Likelihood': 2.11694175436928}

In [75]:
# Comparison with single
eva = EvaluationFramework(IForest(max_features=1))
eva.fit(x)
predictions = eva.predict(x)
eva.score(y,predictions)

{'ROC-AUC': 0.965,
 'Detection-Rate': 1.0,
 'False-Alarm': 0.07,
 'F1-Score': 0.4878048780487805,
 'Log-Likelihood': 2.339777728513415}

### Maximum of Estimators

In [77]:
params_pca = {
    'n_components': [i for i in range(1,x.shape[1]+1)]
}

In [79]:
ens = EnsembleModelParameters(PCA, params_pca)

In [80]:
eva = EvaluationFramework(ens)
eva.fit(x)

In [81]:
predictions = eva.predict(x)
scores = eva.score(y,predictions)

In [84]:
scores

{'ROC-AUC': 0.9666666666666667,
 'Detection-Rate': 1.0,
 'False-Alarm': 0.06666666666666667,
 'F1-Score': 0.5,
 'Log-Likelihood': 2.2283597414413476}

In [85]:
# Comparison with single
eva = EvaluationFramework(PCA())
eva.fit(x)
predictions = eva.predict(x)
eva.score(y,predictions)

{'ROC-AUC': 0.965,
 'Detection-Rate': 1.0,
 'False-Alarm': 0.07,
 'F1-Score': 0.4878048780487805,
 'Log-Likelihood': 2.339777728513415}

### LSCP: Locally Selective Combination of Parallel Outlier Ensembles

Subsets the data into 'local regions' and chooses one of the potential detectors for each region.

In [92]:
selected_detectors = [IForest(), KNN(), PCA(), OCSVM()]
model = LSCP(selected_detectors)

In [93]:
eva = EvaluationFramework(model)
eva.fit(x)

C:\Users\Michael\miniconda3\envs\anomaly\lib\site-packages\pyod\models\lscp.py:383: UserWarning: The number of histogram bins is greater than the number of classifiers, reducing n_bins to n_clf.
  warnings.warn(


In [94]:
predictions = eva.predict(x)
scores = eva.score(y,predictions)
scores

{'ROC-AUC': 0.9733333333333333,
 'Detection-Rate': 1.0,
 'False-Alarm': 0.05333333333333334,
 'F1-Score': 0.5555555555555556,
 'Log-Likelihood': 1.7826877931530787}

### Feature Bagging

In [107]:
base_estimator=PCA()
model = FeatureBagging(base_estimator=base_estimator, n_estimators=10)

In [108]:
eva = EvaluationFramework(model)
eva.fit(x)

In [109]:
predictions = eva.predict(x)
scores = eva.score(y,predictions)
scores

{'ROC-AUC': 0.965,
 'Detection-Rate': 1.0,
 'False-Alarm': 0.07,
 'F1-Score': 0.4878048780487805,
 'Log-Likelihood': 2.339777728513415}